<a href="https://colab.research.google.com/github/SiddanshChawla/Sentiment-Analysis/blob/main/nlp_neural_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv('all-data.csv', encoding='iso-8859-1', names=['sentiment', 'text'])

data.head()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download("stopwords")
nltk.download('punkt')

stopwords = set(stopwords.words('english'))

def process_text(text):
    pattern = '[^a-zA-Z]+'
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [re.sub(pattern, '', token) for token in tokens]
    return tokens

data['tokens'] = data['text'].apply(process_text)

data.head()

data.info()

from sklearn.model_selection import train_test_split
import random

x_train, x_test, y_train, y_test = train_test_split(data['tokens'], data['sentiment'], test_size=0.2, random_state=42)

def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

all_words = nltk.FreqDist(word for doc in x_train for word in doc)
word_features = list(all_words.keys())[:1000]

train_features = [(document_features(d, word_features), s) for d, s in zip(x_train, y_train)]
test_features = [(document_features(d, word_features), s) for d, s in zip(x_test, y_test)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  4846 non-null   object
 1   text       4846 non-null   object
 2   tokens     4846 non-null   object
dtypes: object(3)
memory usage: 113.7+ KB


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Preprocess the data
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
padded_sequences = pad_sequences(sequences, maxlen=100)

In [ ]:
import numpy as np

In [ ]:
label_map = {'positive': 1, 'negative': -1, 'neutral': 0}

# Convert the list of strings to a list of numeric values
numeric_labels = [label_map[label] for label in data['sentiment']]
list_input = np.array(numeric_labels)

In [ ]:
# Split the data into training and testing sets
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(padded_sequences, list_input, test_size=0.2, random_state=42)

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=100))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_rnn, y_train_rnn, epochs=10, batch_size=32)

Epoch 1/10
122/122 [==============================] - 24s 175ms/step - loss: 0.4285 - accuracy: 0.5939
Epoch 2/10
122/122 [==============================] - 23s 184ms/step - loss: -0.1497 - accuracy: 0.6656
Epoch 3/10
122/122 [==============================] - 25s 203ms/step - loss: -1.3297 - accuracy: 0.7291
Epoch 4/10
122/122 [==============================] - 24s 195ms/step - loss: -2.3856 - accuracy: 0.7539
Epoch 5/10
122/122 [==============================] - 22s 184ms/step - loss: -3.4516 - accuracy: 0.7755
Epoch 6/10
122/122 [==============================] - 21s 175ms/step - loss: -3.8060 - accuracy: 0.7469
Epoch 7/10
122/122 [==============================] - 23s 185ms/step - loss: -4.8269 - accuracy: 0.7781
Epoch 8/10
122/122 [==============================] - 23s 186ms/step - loss: -5.5376 - accuracy: 0.7773
Epoch 9/10
122/122 [==============================] - 21s 174ms/step - loss: -6.6489 - accuracy: 0.7918
Epoch 10/10
122/122 [==============================] - 22s 184ms/

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Define CNN model architecture
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

In [ ]:

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train_rnn, y_train_rnn, batch_size=32, epochs=10, validation_data=(X_test_rnn, y_test_rnn))


Epoch 1/10
122/122 [==============================] - 3s 13ms/step - loss: 0.4607 - accuracy: 0.5929 - val_loss: 0.4738 - val_accuracy: 0.5887
Epoch 2/10
122/122 [==============================] - 1s 12ms/step - loss: 0.2786 - accuracy: 0.6192 - val_loss: 0.1615 - val_accuracy: 0.6825
Epoch 3/10
122/122 [==============================] - 2s 14ms/step - loss: -0.3143 - accuracy: 0.6994 - val_loss: -0.4811 - val_accuracy: 0.6742
Epoch 4/10
122/122 [==============================] - 2s 20ms/step - loss: -2.4210 - accuracy: 0.7294 - val_loss: -3.0075 - val_accuracy: 0.6835
Epoch 5/10
122/122 [==============================] - 1s 12ms/step - loss: -8.4017 - accuracy: 0.7557 - val_loss: -9.1802 - val_accuracy: 0.6722
Epoch 6/10
122/122 [==============================] - 1s 12ms/step - loss: -20.5464 - accuracy: 0.7593 - val_loss: -19.5596 - val_accuracy: 0.6680
Epoch 7/10
122/122 [==============================] - 1s 12ms/step - loss: -41.1963 - accuracy: 0.7652 - val_loss: -36.2401 - val_ac